# core

> The core module provides

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| hide
import pandas as pd

In [1]:
class MLProject:
    def __init__(self) -> None:
        pass

In [ ]:
class SemanticDataFrame:
    def __init__(self, df:pd.DataFrame, semantics:dict) -> None:
        self.df = df
        self._sem_vars = [key for key in semantics['variables'].keys()]